## 1. Setup and Imports

In [ ]:
# Add package to path
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import psutil
import os
import time

import matplotlib.pyplot as plt
import h5py

project_root = Path.cwd().parent
if str(project_root / 'src') not in sys.path:
    sys.path.insert(0, str(project_root / 'src'))


## 2. Memory Monitoring Utilities

In [3]:
def get_memory_usage():
    """Get current process memory usage in GB."""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024**3  # Convert to GB

def print_memory_stats(label):
    """Print current memory usage with a label."""
    mem_gb = get_memory_usage()
    print(f"{label}: {mem_gb:.2f} GB")
    return mem_gb

# Baseline memory
baseline_mem = print_memory_stats("Baseline")

Baseline: 0.13 GB


## 3. Define Fields to Extract

Select essential fields for analysis to minimize memory usage.

In [4]:
# Essential fields for most analyses
FIELDS = [
    'mstars_disk',
    'mstars_bulge', 
    'mhhalo',
    'type',
    'x',
    'y', 
    'z',
    'sfr_disk',
    'sfr_burst',
    'zgas_disk',
    'zcold'
]

print(f"Will extract {len(FIELDS)} fields: {FIELDS}")

Will extract 11 fields: ['mstars_disk', 'mstars_bulge', 'mhhalo', 'type', 'x', 'y', 'z', 'sfr_disk', 'sfr_burst', 'zgas_disk', 'zcold']


## 4. Load Data from One Snapshot

Start with a single snapshot to estimate memory requirements.

In [6]:
from galform_analysis.config import get_base_dir

# Get base directory
base_dir = get_base_dir()
print(f"Base directory: {base_dir}")

# Select a snapshot to analyze
snapshot = 'iz201'  # Change this to your desired snapshot
snapshot_path = base_dir / snapshot

print(f"\nLoading snapshot: {snapshot}")
print(f"Path: {snapshot_path}")
print(f"Path exists: {snapshot_path.exists()}")

# Define function to load snapshot into DataFrame
def load_snapshot_to_dataframe(snapshot_path, fields=None, use_float32=True):
    """Load a GALFORM snapshot into a pandas DataFrame."""
    if fields is None:
        fields = FIELDS
    
    all_data = {field: [] for field in fields}
    
    # Iterate over subvolumes
    subvol_dirs = sorted([d for d in snapshot_path.iterdir() if d.is_dir() and d.name.startswith('ivol')])
    
    for subvol_dir in subvol_dirs:
        gal_file = subvol_dir / 'galaxies.hdf5'
        if not gal_file.exists():
            continue
        
        try:
            with h5py.File(gal_file, 'r') as f:
                # Check completion flag
                if 'Output001' not in f:
                    continue
                output_group = f['Output001']
                
                # Check if there are any galaxies
                first_field = fields[0]
                if first_field not in output_group:
                    continue
                
                n_gal = len(output_group[first_field])
                if n_gal == 0:
                    continue
                
                # Load all requested fields
                for field in fields:
                    if field in output_group:
                        arr = output_group[field][:]
                        if use_float32 and arr.dtype in [np.float64, np.float32]:
                            arr = arr.astype(np.float32)
                        all_data[field].append(arr)
                    else:
                        # Fill with NaN if field missing
                        all_data[field].append(np.full(n_gal, np.nan))
        except Exception as e:
            print(f"Warning: Could not read {gal_file}: {e}")
            continue
    
    # Concatenate all subvolumes
    df_data = {}
    for field in fields:
        if len(all_data[field]) > 0:
            df_data[field] = np.concatenate(all_data[field])
        else:
            df_data[field] = np.array([])
    
    return pd.DataFrame(df_data)

# Load the snapshot
start_time = time.time()
df = load_snapshot_to_dataframe(snapshot_path)
load_time = time.time() - start_time

after_load_mem = print_memory_stats("After loading")

print(f"\nLoaded {len(df):,} galaxies in {load_time:.2f} seconds")
print(f"Memory usage: {after_load_mem:.2f} GB")
print(f"Memory increase: {after_load_mem - baseline_mem:.2f} GB")


Base directory: /cosma5/data/durham/dc-hick2/Galform_Out/L800/gp14

Loading snapshot: iz201
Path: /cosma5/data/durham/dc-hick2/Galform_Out/L800/gp14/iz201
Path exists: False


FileNotFoundError: [Errno 2] No such file or directory: '/cosma5/data/durham/dc-hick2/Galform_Out/L800/gp14/iz201'

## 5. Inspect the Data

In [ ]:
# Show data types
print("Data types:")
print(df.dtypes)

# Show first few rows
print("\nFirst 5 rows:")
df.head()

Data types:
Series([], dtype: object)

First 5 rows:


""


## 6. Compute Derived Quantities

Add useful derived fields like total stellar mass and SFR.

In [ ]:
# Compute total stellar mass
if 'mstars_disk' in df.columns and 'mstars_bulge' in df.columns:
    df['mstar'] = df['mstars_disk'] + df['mstars_bulge']

# Compute total SFR
if 'sfr_disk' in df.columns and 'sfr_burst' in df.columns:
    df['sfr'] = df['sfr_disk'] + df['sfr_burst']

# Compute sSFR (specific star formation rate)
if 'sfr' in df.columns and 'mstar' in df.columns:
    df['ssfr'] = df['sfr'] / df['mstar']
    df['ssfr'] = df['ssfr'].replace([np.inf, -np.inf], np.nan)

after_derived_mem = print_memory_stats("After computing derived quantities")
print(f"Memory increase from derived fields: {after_derived_mem - after_load_mem:.2f} GB")

print(f"\nUpdated DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

After computing derived quantities: 0.20 GB
Memory increase from derived fields: 0.00 GB

Updated DataFrame shape: (0, 0)
Columns: []


## 7. Quick Analysis Test

Test that we can efficiently analyze the DataFrame.

In [ ]:
# Check if DataFrame has data
if len(df) == 0:
    print("⚠ Warning: DataFrame is empty! No data was loaded.")
    print("This could mean:")
    print("  - The snapshot directory doesn't exist")
    print("  - No subvolumes have CompletionFlag=1")
    print("  - There was an error reading the HDF5 files")
elif 'mstar' not in df.columns:
    print("⚠ Warning: 'mstar' column not found!")
    print("Available columns:", list(df.columns))
    print("Make sure to run cell 13 (Compute Derived Quantities) first.")
else:
    # Filter for valid galaxies
    valid = (df['mstar'] > 0) & np.isfinite(df['mstar'])
    df_valid = df[valid]
    
    print(f"Valid galaxies: {len(df_valid):,} / {len(df):,}")
    
    # Quick histogram of stellar masses
    if len(df_valid) > 0:
        log_mstar = np.log10(df_valid['mstar'])
        
        plt.figure(figsize=(10, 6))
        plt.hist(log_mstar, bins=50, alpha=0.7, edgecolor='black')
        plt.xlabel(r'$\log_{10}(M_*/M_\odot)$')
        plt.ylabel('Count')
        plt.title(f'Stellar Mass Distribution ({snapshot})')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print("\nStellar mass statistics:")
        print(f"  Min: {log_mstar.min():.2f}")
        print(f"  Max: {log_mstar.max():.2f}")
        print(f"  Median: {log_mstar.median():.2f}")
    else:
        print("No valid galaxies with mstar > 0")

⚠ Warning: DataFrame is empty! No data was loaded.
This could mean:
  - The snapshot directory doesn't exist
  - No subvolumes have CompletionFlag=1
  - There was an error reading the HDF5 files


## 8. Estimate Full Dataset Memory

Extrapolate memory requirements for all snapshots.

In [ ]:
# Memory per galaxy
mem_per_galaxy = df.memory_usage(deep=True).sum() / len(df)
print(f"Memory per galaxy: {mem_per_galaxy / 1024:.2f} KB")

# Count total snapshots
all_snapshots = sorted([d for d in base_dir.iterdir() if d.is_dir() and d.name.startswith('iz')])
print(f"\nTotal snapshots found: {len(all_snapshots)}")

# Estimate for all snapshots (assuming similar galaxy counts)
estimated_total_galaxies = len(df) * len(all_snapshots)
estimated_total_memory_gb = (mem_per_galaxy * estimated_total_galaxies) / 1024**3

print("\nEstimated total memory for all snapshots:")
print(f"  Galaxies per snapshot: {len(df):,}")
print(f"  Total galaxies (all snapshots): {estimated_total_galaxies:,}")
print(f"  Estimated memory: {estimated_total_memory_gb:.2f} GB")

# Memory optimization recommendations
print("\nMemory optimization tips:")
print("  1. Using float32 vs float64: ~50% savings")
print("  2. Selecting fewer fields: proportional savings")
print(f"  3. Loading per-snapshot: ~{estimated_total_memory_gb / len(all_snapshots):.2f} GB each")

Memory per galaxy: inf KB

Total snapshots found: 9

Estimated total memory for all snapshots:
  Galaxies per snapshot: 0
  Total galaxies (all snapshots): 0
  Estimated memory: nan GB

Memory optimization tips:
  1. Using float32 vs float64: ~50% savings
  2. Selecting fewer fields: proportional savings
  3. Loading per-snapshot: ~nan GB each

Total snapshots found: 9

Estimated total memory for all snapshots:
  Galaxies per snapshot: 0
  Total galaxies (all snapshots): 0
  Estimated memory: nan GB

Memory optimization tips:
  1. Using float32 vs float64: ~50% savings
  2. Selecting fewer fields: proportional savings
  3. Loading per-snapshot: ~nan GB each


/tmp/ipykernel_3510487/2668045648.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  mem_per_galaxy = df.memory_usage(deep=True).sum() / len(df)
/tmp/ipykernel_3510487/2668045648.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  estimated_total_memory_gb = (mem_per_galaxy * estimated_total_galaxies) / 1024**3


## 9. Performance Comparison

Compare DataFrame approach vs. repeated HDF5 reads for a typical analysis.

In [ ]:
# Test: Compute mass function from DataFrame
print("Testing performance with DataFrame approach...")
start = time.time()

mass_bins = np.arange(8.0, 12.5, 0.2)
valid_mask = (df['mstar'] > 0) & np.isfinite(df['mstar'])
log_mstar = np.log10(df[valid_mask]['mstar'])
counts, _ = np.histogram(log_mstar, bins=mass_bins)

df_time = time.time() - start
print(f"DataFrame computation time: {df_time:.4f} seconds")

# Note: A full comparison would re-read HDF5 files, but we can estimate
print("\nDataFrame approach is ideal for:")
print("  - Multiple analyses on same data")
print("  - Interactive exploration")
print("  - Complex filtering/grouping operations")
print("\nHDF5 approach is better for:")
print("  - Single-use analyses")
print("  - Memory-constrained environments")
print("  - Accessing only a few fields")

Testing performance with DataFrame approach...


KeyError: 'mstar'

## 10. Summary and Recommendations

In [ ]:
print("=" * 60)
print("MEMORY USAGE SUMMARY")
print("=" * 60)
print(f"\nSingle snapshot ({snapshot}):")
print(f"  Galaxies: {len(df):,}")
print(f"  Fields: {len(df.columns)}")
print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
print(f"  Load time: {load_time:.2f} seconds")

print(f"\nEstimated for all {len(all_snapshots)} snapshots:")
print(f"  Total memory: {estimated_total_memory_gb:.2f} GB")
print(f"  Per-snapshot approach: {estimated_total_memory_gb / len(all_snapshots):.2f} GB each")

print("\nRecommendation:")
if estimated_total_memory_gb < 100:
    print("  ✓ Loading all snapshots is FEASIBLE on HPC")
elif estimated_total_memory_gb < 500:
    print("  ⚠ Loading all snapshots may work, but consider per-snapshot approach")
else:
    print("  ✗ Loading all snapshots NOT recommended, use per-snapshot approach")

print("=" * 60)